# Loading the credentials

In [1]:
# Load the Twitter API credentials from the config file
import json
with open('config.json', 'r') as f:
    config = json.load(f)
    consumer_key = config['consumer_key']
    consumer_secret = config['consumer_secret']
    access_token = config['access_token']
    access_token_secret = config['access_token_secret']

# Disabling warnings

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Downloading twitter data

In [4]:
import tweepy
from textblob import TextBlob
import pandas as pd
import os
import csv

# Authenticate with the Twitter API using Tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Define the search query and parameters
query = "volunteering"
count = 100  # maximum number of tweets per request
max_tweets = 10000  # maximum number of tweets to download
since_date = "2022-05-01"  # earliest date to search from
until_date = "2023-05-01"  # latest date to search to

# Open a CSV file to save the tweets to
with open("tweets.csv", "w", encoding="utf-8", newline="") as file:
    writer = csv.writer(file)

    # Write the header row to the CSV file
    writer.writerow(["created_at", "text"])

    # Download the tweets using pagination
    tweets = []
    num_tweets = 0
    for page in tweepy.Cursor(api.search_tweets, q=query, count=count, lang="en", since_id=since_date, until=until_date).pages():
        for tweet in page:
            # Extract the created_at and text fields from the tweet object
            created_at = tweet.created_at.strftime("%Y-%m-%d %H:%M:%S")
            text = tweet.text

            # Write the tweet to the CSV file
            writer.writerow([created_at, text])

            # Increment the tweet count and check if we have reached the maximum number of tweets
            num_tweets += 1
            if num_tweets >= max_tweets:
                break

        if num_tweets >= max_tweets:
            break

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded

# Create sentiment data

In [ ]:
from textblob import TextBlob
# Open the CSV file containing the tweets
with open("tweets.csv", "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    next(reader)  # skip the header row

    # Create a new CSV file to store the sentiment analysis results
    with open("sentiment.csv", "w", newline="", encoding="utf-8") as outfile:
        writer = csv.writer(outfile)
        writer.writerow(["created_at", "text", "polarity", "subjectivity"])

        # Iterate over each tweet in the CSV file
        for row in reader:
            created_at = row[0]
            text = row[1]

            # Perform sentiment analysis on the tweet text using TextBlob
            blob = TextBlob(text)
            polarity = blob.sentiment.polarity
            subjectivity = blob.sentiment.subjectivity

            # Write the sentiment analysis results to the new CSV file
            writer.writerow([created_at, text, polarity, subjectivity])

# Dashboard 

In [ ]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html

# Load the tweet data
tweet_data = pd.read_csv('tweet_data.csv')

# Create a Plotly line chart of the sentiment over time
fig = px.line(tweet_data, x='date', y='polarity', title='Sentiment over Time')

# Create a Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div(children=[
    html.H1(children='Sentiment Analysis Dashboard'),
    html.Div(children='''A dashboard to display sentiment analysis on a specific topic.'''),
    dcc.Graph(id='sentiment-chart', figure=fig)
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)